In [ ]:
#ライブラリをインポート
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import lightgbm as lgb
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from ydata_profiling import ProfileReport

In [ ]:
#データを読み込む
path ="/kaggle/input/titanic/"
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")
df = pd.concat([train , test])
df.info()

In [ ]:
#Nameから敬称を抽出してカウントする
Titles = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
df["Title"] = Titles 

df["Title"].value_counts()

In [ ]:
#敬称をまとめる
df["Title"] = df["Title"].replace(["Mlle","Ms"] ,"Miss")
df["Title"] = df["Title"].replace(["Mme","Lady"] ,"Mrs")
df["Title"] = df["Title"].replace(["Dona" , "Countess" , "Dr","Rev" , "Don" , "Major" , "Col" , "Sir" , "Capt" , "Jonkheer"] ,"Rare")
print(df["Title"].value_counts())
print(df['Title'].info())

In [ ]:
#欠損値を埋める。Fareをとりあえず中央値で埋める。
df["Fare"] = df["Fare"].fillna(df["Fare"].median())
print(df['Fare'].value_counts())
print(plt.hist(data=df , x="Fare", bins=30))

In [ ]:
#その後、対数変換して分布を整える
df['Fare_loge'] = np.log1p(df["Fare"])
print(df['Fare_loge'].value_counts())
print(plt.hist(data=df , x="Fare_loge", bins=30))

In [ ]:
#AgeはTitleごとの中央値で埋める
df["Age"] = df["Age"].fillna(df.groupby("Title")["Age"].transform("median"))
df.isnull().sum()
print(df["Age"].value_counts())
plt.hist(df["Age"], bins=30)

In [ ]:
#Embarkedは最頻値で埋める
#最頻値が複数あるため[0]えｄ愛書の値を取り出す
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0]) 
df.info()

In [ ]:
#特徴量エンジニアリング
#EmbarkedとSexとTitleをワンホットエンコーディングする。
emb = pd.get_dummies(df["Embarked"],prefix="Emb")
sx = pd.get_dummies(df["Sex"],prefix="Sex")
ttl = pd.get_dummies(df["Title"],prefix="Title")

#新しい特徴量として、Famsize,Isalone,IsFareZeroを追加する。
df["Famsize"] = df["SibSp"] + df["Parch"] + 1
df["Isalone"] = df["Famsize"]==1
df["IsFareZero"] = df["Fare"]==0

#追加した特徴量をデータフレームへ統合する。不要なカラムは削除する。
df2 = pd.concat([df, emb, sx, ttl],axis=1)
df3 = df2.drop(columns=["Fare", "Sex_male", "Embarked", "Sex", "Title", "Cabin", "Ticket", "Name"])

df3.head()

In [ ]:
#機械学習モデルに投入するデータを作成する。
train_df = df3[~df3["Survived"].isnull()]
test_df = df3[df3["Survived"].isnull()]

train_x = train_df.drop(columns=["Survived"])
train_y = train_df["Survived"]
train_y.info()

# ↓ランダムフォレスト

In [ ]:
# ランダムフォレストモデル（分類器）の定義
# n_estimators: 使用する決定木の数（一般的に多いほど性能が向上するが、計算時間が増える）
# max_depth: 決定木の深さの最大値（過学習を防ぐために設定することが多い）
rf_model = RandomForestClassifier(
    n_estimators=200,      # 適切な木の本数
    max_depth=7,           # 深さを制限（過学習対策）
    min_samples_split=10,  # 分岐の条件を厳しくする
    min_samples_leaf=5,    # 葉のサイズを大きくする
    random_state=42,
    n_jobs=-1
)

# **↓LithtGBM**

In [ ]:
#機械学習モデルを呼び出し、学習を行う。
lgb_model = lgb.LGBMClassifier(
    objective='binary',
    learning_rate=0.03,    # 低めに設定
    n_estimators=150,      # 試行回数
    num_leaves=8,          # 浅い木に制限（過学習対策）
    max_depth=5,           # 深さの制限
    random_state=42,
)
print("LightGBMのハイパーパラメータを設定しました。")

In [ ]:
#↓どちらのモデルで学習するか、選んでネ！
#rf_model.fit(train_x , train_y)
lgb_model.fit(train_x , train_y)

print("学習が完了しました")

In [ ]:
#交差検証を開始する。5分割がよさそう。
N_SPLITS = 5
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# cross_val_scoreで精度（accuracy）を計算
# scoring='accuracy' はタイタニックの評価指標
accuracy_scores = cross_val_score(
    estimator=lgb_model, 
    X=train_x, 
    y=train_y, 
    cv=skf, 
    scoring='accuracy',
    n_jobs=-1 # 全てのコアを使って並列処理（高速化）
)

# 結果の表示
print(f"各分割での精度スコア ({N_SPLITS}-Fold): {accuracy_scores}")
print(f"----------------------------------------")
print(f"平均精度 (Mean Accuracy): {accuracy_scores.mean():.4f}")
print(f"スコアの標準偏差 (Std Dev): {accuracy_scores.std():.4f}")

In [ ]:
test_df.tail()

In [ ]:
#予測用データの作成をする
test_x = test_df.drop(columns=["Survived"])
test_y = test_df["Survived"]
print("予測用データの作成が完了しました")

In [ ]:
#学習したパラメータをテストデータへ適用して予測する。
test_y = lgb_model.predict(test_x)

#提出用のデータを作る。
submission_df = pd.DataFrame({
    "PassengerId" : test_df["PassengerId"],
    "Survived" : test_y.astype(int)
})

#csvで書き出す
submission_df.to_csv("first_submission13_IsFareZero_farelog_Emb.csv" , index=False)
print("書き出しが完了しました")

In [ ]:
submission_df.info()

In [ ]:
# プロファイルレポートの生成
profile = ProfileReport(train_df, title='My Data Profiling Report')
# HTMLファイルとして保存
profile.to_file("my_report.html")

In [ ]:
farezero =train_df[train_df["Fare"]==0]
farezero

次：
①ランダムフォレストに変えてみる→LithtGBMよりスコアが伸びなそう。
ハイパーパラメータチューニングを頑張ってみる。→結構よい感じになってきた。

②Fare=0の乗客がかなりいる。調べてみると、ほぼ生き残っていないので、新しい特徴量（IsFareZero）を作ってみる。→スコア変わらず。

③Fareの最大値が大きいため、対数変換をして新しい特徴量として加える→スコア変わらず

④Embarkedを特徴量に追加。→そのままではスコアが下がった。→num_leavesとdepthを2ずつ増やしてみた→めっちゃ下がったからもどした